In [35]:
import pandas as pd
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler

import plotly
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

import webbrowser
from threading import Timer

In [ ]:
###read data

In [2]:
data = pd.read_csv("https://firms2.modaps.eosdis.nasa.gov/data/active_fire/noaa-20-viirs-c2/csv/J1_VIIRS_C2_Global_7d.csv")
data.head(20)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight
0,56.62718,161.29944,341.5,0.61,0.71,2020-09-27,6,1,nominal,2.0NRT,286.2,14.2,D
1,56.62842,161.29918,343.0,0.61,0.71,2020-09-27,6,1,nominal,2.0NRT,285.9,12.7,D
2,40.63516,17.98946,308.7,0.43,0.38,2020-09-27,30,1,nominal,2.0NRT,283.9,1.0,N
3,40.63040,17.99883,306.8,0.43,0.38,2020-09-27,30,1,nominal,2.0NRT,283.7,1.0,N
4,40.51973,17.20676,325.5,0.46,0.39,2020-09-27,30,1,nominal,2.0NRT,288.6,3.4,N
5,40.50838,17.20981,317.7,0.46,0.39,2020-09-27,30,1,nominal,2.0NRT,286.1,3.5,N
6,36.86728,34.75681,300.7,0.55,0.68,2020-09-27,30,1,nominal,2.0NRT,289.6,1.2,N
7,40.50417,17.21441,300.6,0.45,0.39,2020-09-27,30,1,nominal,2.0NRT,288.3,0.9,N
8,40.50125,17.20830,299.6,0.46,0.39,2020-09-27,30,1,nominal,2.0NRT,286.9,0.9,N
9,36.86781,34.75771,300.8,0.55,0.68,2020-09-27,30,1,nominal,2.0NRT,290.0,0.8,N


In [ ]:
###extract

In [3]:
rdata = data[["longitude","latitude","acq_date","confidence","bright_ti5"]]
rdata["intensity"] = 1.0
for i in rdata.index :
    if rdata["confidence"][i]==("high") :
        rdata["intensity"][i] = 100*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("nominal") :
        rdata["intensity"][i] = 50*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("low") :
        rdata["intensity"][i] = rdata["bright_ti5"][i]
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aa

,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,161.29944,56.62718,2020-09-27,nominal,286.2,14310.0
1,161.29918,56.62842,2020-09-27,nominal,285.9,14295.0
2,17.98946,40.63516,2020-09-27,nominal,283.9,14195.0
3,17.99883,40.63040,2020-09-27,nominal,283.7,14185.0
4,17.20676,40.51973,2020-09-27,nominal,288.6,14430.0


In [4]:
scaler = MinMaxScaler(feature_range=(0,5))
rdata["intensity"] = scaler.fit_transform(rdata[["intensity"]])
rdata["intensity"] = rdata["intensity"].round().astype(int)
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,161.29944,56.62718,2020-09-27,nominal,286.2,2
1,161.29918,56.62842,2020-09-27,nominal,285.9,2
2,17.98946,40.63516,2020-09-27,nominal,283.9,2
3,17.99883,40.63040,2020-09-27,nominal,283.7,2
4,17.20676,40.51973,2020-09-27,nominal,288.6,2


In [30]:
severity=rdata["intensity"].values.tolist()
interp1dobj = interp1d([0,max(severity)],[1,12])
circle=interp1dobj(severity)
rdata["circle_radius"] = circle
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,longitude,latitude,acq_date,confidence,bright_ti5,intensity,circle_radius
0,161.29944,56.62718,2020-09-27,nominal,286.2,2,5.4
1,161.29918,56.62842,2020-09-27,nominal,285.9,2,5.4
2,17.98946,40.63516,2020-09-27,nominal,283.9,2,5.4
3,17.99883,40.63040,2020-09-27,nominal,283.7,2,5.4
4,17.20676,40.51973,2020-09-27,nominal,288.6,2,5.4


In [36]:
###automatically open graph in browser
def open_browser():
      webbrowser.open_new('http://127.0.0.1:8050/')

In [ ]:
###heat map with slider

In [50]:
dates = rdata["acq_date"].unique()
mark_values = {
    1 : dates[0],
    2 : dates[1],
    3 : dates[2],
    4 : dates[3],
    5 : dates[4],
    6 : dates[5],
    7 : dates[6],
    8 : dates[7]
}

In [51]:
app = dash.Dash()

In [52]:
app.layout = html.Div([
    
    html.Div([
        dcc.Graph(id="viirs")
    ]),
    
    html.Div([
        html.Label(["Choose a day to view data"],style={"text-align":"center","font-weight":"bold"}),
    ]),
        
        
    html.Div([
       dcc.Slider(
           id="slider",
           min = 1,
           max = 8,
           value = 1,
           marks = mark_values,
           step = None,
        ), 
    ],style={"width":"70%","position":"absolute","left":"5%"}),
        
])

In [53]:
@app.callback(
    Output('viirs','figure'),
    [Input('slider','value')],
)

def update_graph(day_choice):
    day_choice = mark_values[day_choice]
    trace = rdata.loc[rdata["acq_date"]==day_choice]
    heat_fig = px.density_mapbox(trace.head(100000),
                                 lat="latitude",
                                 lon="longitude",
                                 radius=circle_radius,
                                 zoom=1,
                                 mapbox_style="open-street-map",
                                )
    return(heat_fig)

In [54]:
Timer(0, open_browser).start()
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
# carto-positron
# carto-darkmatter
# stamen-terrain
# stamen-toner
# stamen-watercolor